# Import

In [1]:
import pandas as pd
import numpy as np
import random

# Function

In [2]:
### Get points
def get_points(data='att48.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()
#     t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone
    ### Denominator
    tsa = [[t0[i][j]**alpha for i in range(all_points)] for j in range(all_points)] # t square alpha
    nsb = [[n[i][j]**beta for i in range(all_points)] for j in range(all_points)] # n square beta
    txn = [[tsa[i][j] * nsb[i][j] for i in range(all_points)] for j in range(all_points)] # tsa * nsb
    denominator = sum(sum(np.array(txn))) # sum(np.array(txn) : Sum of fist row, # sum(sum(np.array(txn)) : Sum of all rows

In [3]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 3
ants= 48
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone
### Denominator
tsa = [[t0[i][j]**alpha for i in range(all_points)] for j in range(all_points)] # t square alpha
nsb = [[n[i][j]**beta for i in range(all_points)] for j in range(all_points)] # n square beta
txn = [[tsa[i][j] * nsb[i][j] for i in range(all_points)] for j in range(all_points)] # tsa * nsb
denominator = sum(sum(np.array(txn))) # sum(np.array(txn) : Sum of fist row, # sum(sum(np.array(txn)) : Sum of all rows

list_gens = []
list_ants = []
list_total_distance = {}
list_paths = {}
list_pheromones = {}
for gen in range(gens):
    print(f'Gen : {gen}')


    
    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### Start
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        
        ### Probability
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0


        ### Total distance
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])
        print(f'Total Distance : {total_distance}')

        list_gens.append(gen)
        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### Cost matrix reset
        reset_values()

    ### Update pheromone
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
#                     else:
#                         t0[i][j] = 0
                except:
                    pass
    # list_pheromones.update({gen : t0})

#     print(f't0 of ant {gen+1} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    print(f'-------------')

    ### Pheromone reset
    # reset_values()

# print(list_pheromones)
# return [list_gens, list_ants, list_total_distance, list_paths]
    

Gen : 0
Ant : 0
Total Distance : 37058.28838230712
Ant : 1
Total Distance : 35851.504269928904
Ant : 2
Total Distance : 37225.22265046211
Ant : 3
Total Distance : 41018.91208627347
Ant : 4
Total Distance : 38126.349279601476
Ant : 5
Total Distance : 36726.88434604734
Ant : 6
Total Distance : 39309.04802288733
Ant : 7
Total Distance : 40948.54049952079
Ant : 8
Total Distance : 42268.12149826158
Ant : 9
Total Distance : 38355.86967514288
Ant : 10
Total Distance : 36982.62895589434
Ant : 11
Total Distance : 35135.455097450664
Ant : 12
Total Distance : 35340.486700321555
Ant : 13
Total Distance : 37960.19448948187
Ant : 14
Total Distance : 38139.81128027202
Ant : 15
Total Distance : 39964.11686816355
Ant : 16
Total Distance : 38703.24564638667
Ant : 17
Total Distance : 35512.194487850786
Ant : 18
Total Distance : 36393.11464341938
Ant : 19
Total Distance : 36921.659400007324
Ant : 20
Total Distance : 37205.4883208474
Ant : 21
Total Distance : 37239.225898352044
Ant : 22
Total Distance : 41